In [56]:
pip install cleantext

In [57]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopy
import folium
from sklearn.preprocessing import StandardScaler
from geopy.geocoders import Nominatim, GoogleV3
from folium.plugins import MarkerCluster
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import nltk
from textblob import TextBlob
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from textblob import TextBlob
import re
import nltk
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from cleantext import clean as cleaner
from sklearn.decomposition import TruncatedSVD

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
order = pd.read_csv('/content/drive/My Drive/data.csv')

In [60]:
order.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,customer_id,Country,review_score,product_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 8:26,2.55,17850.0,United Kingdom,5,1376722
1,536365,71053,WHITE METAL LANTERN,6,12-01-2010 8:26,3.39,17850.0,United Kingdom,4,1440895
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 8:26,2.75,17850.0,United Kingdom,1,1842335
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 8:26,3.39,17850.0,United Kingdom,1,1556721
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 8:26,3.39,17850.0,United Kingdom,2,1057706


In [61]:
order_rating = order.loc[:, ['customer_id', 'product_id', 'review_score']]

In [62]:
order_rating.head()

,customer_id,product_id,review_score
0,17850.0,1376722,5
1,17850.0,1440895,4
2,17850.0,1842335,1
3,17850.0,1556721,1
4,17850.0,1057706,2


In [63]:
order_rating = order_rating[order_rating['product_id'].isin(
    order_rating['product_id'].value_counts()[
        order_rating['product_id'].value_counts() > 1].index)]

In [64]:
order_rating.head()

,customer_id,product_id,review_score
27,12583.0,1804535,3
32,12583.0,1554483,1
201,12431.0,1579915,4
338,15862.0,1492546,3
368,15862.0,1507846,1


In [65]:
order_rating = order_rating.reset_index()

In [66]:
ratings_utility_matrix = order_rating.pivot_table(values='review_score',
                                                  index='customer_id',
                                                  columns='product_id',
                                                  fill_value=0)
ratings_utility_matrix.head()

product_id,1000583,1012210,1016379,1059629,1082077,1089904,1105514,1119968,1137956,1141682,...,1809346,1823076,1827867,1828184,1889837,1897902,1903838,1918812,1930461,1999435
customer_id,,,,,,,,,,,,,,,,,,,,,
12431.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12433.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12472.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
12583.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12720.0,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0


In [67]:
ratings_utility_matrix.shape

(66, 56)

In [68]:
X = ratings_utility_matrix.T
X.head()

customer_id,12431.0,12433.0,12472.0,12583.0,12720.0,12725.0,12838.0,12971.0,13013.0,13069.0,...,17238.0,17243.0,17567.0,17581.0,17787.0,17850.0,17884.0,17976.0,18011.0,18041.0
product_id,,,,,,,,,,,,,,,,,,,,,
1000583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1012210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1016379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
1059629,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,0
1082077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
X.shape

(56, 66)

In [70]:
def fitsystemrecommendation(ratings_utility_matrix):
    X = ratings_utility_matrix.T
    SVD = TruncatedSVD(n_components=10)
    decomposed_matrix = SVD.fit_transform(X)
    correlation_matrix = np.corrcoef(decomposed_matrix)
    return correlation_matrix

In [71]:
correlation_matrix = fitsystemrecommendation(ratings_utility_matrix)

In [74]:
order_rating

,index,customer_id,product_id,review_score
0,27,12583.0,1804535,3
1,32,12583.0,1554483,1
2,201,12431.0,1579915,4
3,338,15862.0,1492546,3
4,368,15862.0,1507846,1
...,...,...,...,...
114,9446,14506.0,1420703,4
115,9456,14506.0,1752958,4
116,9474,14506.0,1186531,4
117,9602,15889.0,1473532,2


In [72]:
def systemrecommendation(prod_id):
    order_rating.index[order_rating['product_id'] == prod_id].tolist()[1]
    product_names = list(X.index)
    product_ID = product_names.index(prod_id)
    correlation_product_ID = correlation_matrix[product_ID]
    Recommend = list(X.index[correlation_product_ID > 0.70])
    Recommend.remove(prod_id)
    return pd.DataFrame(Recommend[0:9], columns=['Recommendation'])

In [75]:
systemrecommendation(1752958)

,Recommendation
0,1186531
1,1368078
2,1420703
